# Vanilla echo-state network

The goal of this exercise is to get some intuition about the functioning of echo-state networks.

Uncomment this line to install ANNarchy if you haven't already:

In [ ]:
#!pip install ANNarchy


Let's start by importing numpy, matplotlib and ANNarchy:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import ANNarchy as ann

Recurrent ESN neurons are defined by a variable $x$ following this ODE:

$$
    \tau \, \dfrac{d x}{dt} + x = \sum_\text{inputs} w_i \, r_i + \sum_\text{recurrent} w_i \, r_i
$$

where $\tau$ is a time constant. They integrate two sorts of inputs, coming from input neurons and the other recurrent neurons.

The firing rate $r$ of a neuron applies the `tanh` function on $x$:

$$
    r = \tanh{x}
$$

The following cell defines such a neuron in ANNarchy:

In [ ]:
ESN_Neuron = ann.Neuron(
    parameters = """
        tau = 30.0     : population   # Time constant
        g = 1.0        : population   # Scaling
    """,
    equations="""
        tau * dx/dt + x = sum(in) + g * sum(exc)
        r = tanh(x)
    """
)

An ESN is typically composed of an input population, the reservoir with random recurrent connections and a readout population linearly reading the reservoir's activity:

$$
    \mathbf{z} = W^\text{out} \times \mathbf{r}
$$

The following class defines an input population with a single neuron and a reservoir of `N` neurons. The readout population will be implemented externally.

The weights between the input and recurrent populations are randomly drawn from $\mathcal{N}(0, 1)$. The recurrent weights are drawn from $\mathcal{N}(0, \dfrac{1}{\sqrt{N}})$.

The scaling factor $g$ is integrated in the neuron definition, what allows to vary it easily without redrawing the random weights.

The `trial()` method allows to simulate the ESN for a single trial. After a reset period of 100 ms, an impulse is set in the input population for 100 ms. The ESN relayes for the rest of the trial (duration - 200 ms). The monitor records the firing rate `r` in the recurrent population and returns it. 

In [ ]:
class ESN:
    """
    Echo-state network implemented in ANNarchy.
    """
    def __init__(self, N:int):
        """
        Args:
            N : number of neurons in the reservoir
        """
        self.N = N

        # Clear ANNarchy to allow multiple instances.
        ann.clear()

        # Input population
        self.inp = ann.Population(1, ann.Neuron(parameters="r=0.0"))

        # Recurrent population
        self.reservoir = ann.Population(N, ESN_Neuron)

        # Input weights
        self.Wi = ann.Projection(self.inp, self.reservoir, 'in')
        self.Wi.connect_all_to_all(weights=ann.Normal(0.0, 1.0))

        # Recurrent weights
        self.Wrec = ann.Projection(self.reservoir, self.reservoir, 'exc')
        self.Wrec.connect_all_to_all(weights=ann.Normal(0., 1/np.sqrt(N)))

        # Monitor
        self.monitor = ann.Monitor(self.reservoir, 'r')

        ann.compile()

    def trial(self, g:float, duration:int=5000, amplitude:float=1.0):
        """
        Runs a single trial for a given spectral radius.
        
        Args:
            g: scaling factor.
            duration: duration of the trial.
            amplitude: amplitude of the impulse.
        """

        # Reset and set spectral radius
        self.inp.r = 0.0
        self.reservoir.x = 0.0
        self.reservoir.r = 0.0
        self.reservoir.g = g
        
        # 100 ms of reset
        ann.simulate(100.)
        
        # 100 ms of stimulation
        self.inp.r = amplitude
        ann.simulate(100.0)
        
        # Relax for the rest of the duration
        self.inp.r = 0.0
        ann.simulate(duration - 200.)
        
        # Return the firing rates r
        data = self.monitor.get('r')    
        return data

In [ ]:
net = ESN(N=500)

**Q:** Run the network for multiple values of $g$ for 5 seconds, with an impulse of amplitude 1.0. Plot the firing rate of a handful of neuron (e.g. 5) and observe the influence of $g$ on the dynamics. When do complex dynamics appear?

**Q:** For different values of $g$, run two trials of the same network: one where the amplitude of the impulse is 1.0, the other where it is very slightly different (e.g. 1.00001). Plot the difference between the two runs. When does chaos appear? Do not hesitate to simulate for longer.

Let's now use the reservoir as a spatiotemporal basis for supervised learning. We first need to define a target signal $t$ that will be used to train the weights of the readout population. We can use for example an impulse after 4 seconds as a target, but feel free to use whatever you want:

In [ ]:
target = np.zeros(5000)
target[4000:4500] = 1.0

plt.figure(figsize=(10, 6))
plt.plot(target)
plt.show()

Instead of implementing a readout neuron and a linear regression / delta learning rule, we are going to make use of the `scikit-learn`'s linear regression implementation. 

If you have an input array `X` and a target array `t` (the first dimension of both arrays being samples), you can simply type:

```python
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, t)
```

Ridge regression (<https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html>) is another popular option.

You can then predict value for any other input array `X`:

```python
y = reg.predict(X)
```

**Q:** Train the linear regression algorithm on the activity of the reservoir as input. Samples are the firing rates of the reservoir at each time step (i.e. 5000 samples of dimension `N=500`). Test it on the same array and visualize the prediction. Can we linearly predict the target signal using an ESN? 


**Q:** Test the learned regression on the activity of the reservoir, but using an input impulse of slightly different amplitude (e.g. 1.00001). Vary $g$. Does it still work? Conclude on the stability and usability of ESN reservoirs.